# GEC Model Training

## Imports and Setup

In [2]:
%load_ext autoreload
%autoreload 2

!mkdir cfgs
!mkdir processed_data

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import pandas as pd
import pickle
import os
from utils import Config, Training_config

main_args = Config()
!mkdir cfgs
!mkdir processed_data

mkdir: cannot create directory ‘cfgs’: File exists
mkdir: cannot create directory ‘processed_data’: File exists


In [ ]:
!pip install simpletransformers

In [5]:
from simpletransformers.ner import NERModel

In [6]:
output_base_dir = os.path.join(os.path.abspath('.'), 'outputs')

print(f'Loading config from {main_args.ECC_TRAIN_CONFIG}')
args = Training_config(main_args.ECC_TRAIN_CONFIG)
labels = args.BIO_labels

if args.model_type != 'interactive':
    labels = labels[:-1]

train_file, eval_file, test_file = args.train_dev_data

print("reading train file:", train_file)
with open(train_file, 'rb') as f:
    train_data = pickle.load(f)
print("reading eval file:", eval_file)
with open(eval_file, 'rb') as f:
    eval_data = pickle.load(f)
print("reading test file:", test_file)
with open(test_file, 'rb') as f:
    test_data = pickle.load(f)

if args.debug:
    train_data = train_data[:[i for i, d in enumerate(train_data) if d[0]<6][-1]]
    eval_data = eval_data[:[i for i, d in enumerate(eval_data) if d[0]<6][-1]]
    test_data = test_data[:[i for i, d in enumerate(test_data) if d[0]<6][-1]]
    wandb_project = False
else:
    wandb_project = "gec_ner"

columns=["sentence_id", "words", "labels", "cls_labels", "correction_index", "parsing_embedding"]

train_df = pd.DataFrame(train_data, columns=columns)
eval_df = pd.DataFrame(eval_data, columns=columns)
if test_file:
    test_df = pd.DataFrame(test_data, columns=columns)
if not args.parsing_embedding:
    train_df = train_df.drop(['parsing_embedding'], axis=1)
    eval_df = eval_df.drop(['parsing_embedding'], axis=1)
    test_df = test_df.drop(['parsing_embedding'], axis=1)

print(len(train_df), len(eval_df))
print(len(test_df))

if args.only_inference is not None:
    args.model_name = output_base_dir + args.exp_name
print(args.exp_name)

if args.only_inference is not None:
    if args.output_dir is None:
        output_dir = output_base_dir + args.exp_name
    else:
        output_dir = args.output_dir
        args.exp_name = output_dir
else:
    output_dir = output_base_dir + args.exp_name + '/eval'

model_args = {"overwrite_output_dir": True,
          "num_train_epochs": args.epochs,
          "train_batch_size": args.train_batch_size,
          "eval_batch_size": args.eval_batch_size,
          "output_dir": output_dir,
          "reprocess_input_data": True,
          "special_tokens_list": ["[NONE]", "[MOD]"],
          "wandb_kwargs": {
              "mode": 'online', #'offline',
              "name": args.exp_name,
          },
          "wandb_project": wandb_project,
          "evaluate_during_training": args.evaluate_during_training,
          "evaluate_each_epoch": args.evaluate_each_epoch,
          "learning_rate": args.lr,
          "multi_loss": args.multi_loss,
          "wo_token_labels": args.wo_token_labels,
          "cls_num_labels": 15, # label nums for [CLS] token classification
          "use_multiprocessing_for_evaluation": False,
          "use_multiprocessing": args.use_multiprocessing,
          "loss_weight": args.loss_weight,
          "max_correction_embeddings": args.max_correction_embeddings,
          "max_seq_length": args.max_seq_length,
          "n_gpu": args.n_gpu,
          "dataloader_num_workers": 20,
          "save_eval_checkpoints": False,
          "early_stopping_metric": "f1_score",
          "best_model_dir": output_dir,
          "parsing_embedding": args.parsing_embedding,
          "parsing_embedding_for_embedding": args.parsing_embedding_for_embedding,
          "logging_steps": 0,
          "manual_seed": 42
          }

model = NERModel(
    model_type=args.model,
    model_name=args.model_name,
    labels=labels,
    args=model_args,
    use_cuda=args.use_cuda
)

# Create a NERModel
if args.only_inference is None:
    # Train the model
    model.train_model(train_df, eval_data=eval_df, test_data=test_df)

# if args.only_inference is None or 'dev' in args.only_inference:
#     result, model_outputs, predictions, out_label_list = model.eval_model(eval_df, wandb_log=False)
# if args.only_inference is None or 'test' in args.only_inference:
#     result, model_outputs, predictions, out_label_list = model.eval_model(test_df, wandb_log=False)

Loading config from cfgs/train_error_correction_ce_syntax.py
reading train file: processed_data/bert_train.pkl
reading eval file: processed_data/bert_dev.pkl
reading test file: processed_data/bert_test.pkl
130039 16452
16373
/error_correction_ce_syntax/ner_1e5_bs32_bert_large


Some weights of BertForTokenClassification were not initialized from the model checkpoint at espejelomar/beto-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


/usr/local/lib/python3.10/dist-packages/simpletransformers/ner/ner_utils.py:190: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  return [


  0%|          | 0/4153 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 20 worker processes in total. Our suggested max number of worker in current system is 8, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


...

  0%|          | 0/520 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/9 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:159: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(
/usr/local/lib/python3.10/dist-packages/simpletransformers/ner/ner_utils.py:190: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to

  0%|          | 0/519 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:159: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(


In [ ]:
# result, model_outputs, predictions, out_label_list

result, model_outputs, predictions = model.eval_model(eval_df, wandb_log=False)

/usr/local/lib/python3.10/dist-packages/simpletransformers/ner/ner_utils.py:190: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  return [


  0%|          | 0/2413 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/38 [00:00<?, ?it/s]

In [ ]:
model.args.labels_list

['B-s1',
 'I-s1',
 'B-s2',
 'I-s2',
 'B-s3',
 'I-s3',
 'B-s5',
 'I-s5',
 'B-s6',
 'I-s6',
 'B-s7',
 'I-s7',
 'B-s8',
 'I-s8',
 'B-s9',
 'I-s9',
 'B-m1',
 'I-m1',
 'B-m2',
 'I-m2',
 'B-m3',
 'I-m3',
 'B-m4',
 'I-m4',
 'B-m5',
 'I-m5',
 'B-m6',
 'I-m6',
 'B-m7',
 'I-m7',
 'B-oth',
 'I-oth',
 'O']

In [7]:
#Save model
model.model.save_pretrained('gec_cows_l2h_small')
model.tokenizer.save_pretrained('gec_cows_l2h_small')
model.config.save_pretrained('gec_cows_l2h_small/')

In [11]:
# Reload saved model and try inference
reloaded = NERModel(model_type=args.model,model_name='gec_cows_l2h_small', labels=labels)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [17]:
eval_df

,sentence_id,words,labels,cls_labels,correction_index,parsing_embedding
0,0,mis,O,s5,1,1
1,0,primas,O,s5,0,2
2,0,y,O,s5,0,0
3,0,yo,O,s5,0,0
4,0,vamos,O,s5,0,0
...,...,...,...,...,...,...
16447,519,Ella,O,s2,0,0
16448,519,tiene,O,s2,0,0
16449,519,muchas,O,s2,0,3
16450,519,ventajas,O,s2,0,0


In [12]:
result, model_outputs, predictions = reloaded.eval_model(eval_df[0:2], wandb_log=False)

/usr/local/lib/python3.10/dist-packages/simpletransformers/ner/ner_utils.py:190: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  return [


  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/1 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:159: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(


In [ ]:
result

{'eval_loss': 0.2679738557260287,
 'precision': 0.49108423857347816,
 'recall': 0.5652276480301958,
 'f1_score': 0.5255538495284053}

In [ ]:
predictions

In [19]:
reloaded.predict(["Hola que es tal"])

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

([[{'Hola': 'O'}, {'que': 'O'}, {'es': 'O'}, {'tal': 'O'}]],
 [[{'Hola': [[-0.9136,
      -0.835,
      -1.4375,
      -1.131,
      -1.327,
      -0.6626,
      -0.8394,
      -1.448,
      -1.825,
      -1.351,
      -1.601,
      -0.6978,
      -1.449,
      -0.9297,
      -0.8047,
      -1.4795,
      -1.117,
      -1.69,
      -0.8022,
      -1.565,
      -1.101,
      -1.272,
      -1.857,
      -1.404,
      -0.8965,
      -1.129,
      -1.518,
      -1.472,
      -1.201,
      -1.564,
      -1.584,
      -1.549,
      12.03]]},
   {'que': [[-1.038,
      -0.813,
      -1.422,
      -1.161,
      -1.392,
      -0.719,
      -0.7607,
      -1.391,
      -1.886,
      -1.307,
      -1.471,
      -0.7275,
      -1.416,
      -0.92,
      -0.883,
      -1.454,
      -1.12,
      -1.55,
      -0.7275,
      -1.537,
      -1.141,
      -1.441,
      -1.855,
      -1.304,
      -0.8286,
      -1.036,
      -1.558,
      -1.594,
      -1.269,
      -1.628,
      -1.579,
      -1.519,
  

In [23]:
!ls -al gec_cows_l2h_small/

ls: cannot access 'gec_cows_l2h_small/': No such file or directory


In [22]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
